<a href="https://colab.research.google.com/github/shin0105/4YP/blob/master/RFregression_Boston.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bayesian-optimization

In [18]:
#import packages
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold
from sklearn import ensemble
from sklearn.metrics import mean_squared_error,accuracy_score
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import matplotlib.pyplot as plt
from bayes_opt import BayesianOptimization

In [19]:
#reading csv file for Boston Housing Dataset
from google.colab import files
uploaded = files.upload()
import io
df = pd.read_csv(io.BytesIO(uploaded['hou_all.csv']),\
                 names = ['CRIM','ZN','INDUS','CHAS','NOS','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT','MEDV'])
df.head()

Saving hou_all.csv to hou_all (1).csv


,CRIM,ZN,INDUS,CHAS,NOS,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [20]:
#parameters = CRIM , ZN, INDUS
X = df.iloc[:,0:3].to_numpy()

#target = MEDV
Y = df.iloc[:,-1].to_numpy()

#train test split
xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size = 0.3, random_state = 10)

#building rf regression model
rf = RandomForestRegressor()
rf.fit(xTrain,yTrain)
yPred = rf.predict(xTest)

#calculating loss - MSE
MSE = mean_squared_error(yPred,yTest)

In [30]:
def bo_params_rf(max_samples,n_estimators,max_features):
    
    params = {
        'max_samples': max_samples,
        'max_features':max_features,
        'n_estimators':int(n_estimators)
    }
    clf = RandomForestRegressor(max_samples=params['max_samples'],max_features=params['max_features'],n_estimators=params['n_estimators'])
    rf.fit(xTrain,yTrain)
    yPred = rf.predict(xTest)
    score = -mean_squared_error(yPred,yTest)
    return score

In [31]:
bounds = {'max_samples':(0.5,1),
          'max_features':(0.5,1),
          'n_estimators':(100,200)}

rf_bo = BayesianOptimization(bo_params_rf, bounds)

results = rf_bo.maximize(n_iter=50, init_points=10)

|   iter    |  target   | max_fe... | max_sa... | n_esti... |
-------------------------------------------------------------
| 1         | -68.18    | 0.9851    | 0.5674    | 191.5     |
| 2         | -66.19    | 0.9668    | 0.9721    | 171.7     |
| 3         | -65.54    | 0.6616    | 0.7355    | 165.3     |
| 4         | -65.85    | 0.728     | 0.5537    | 137.7     |
| 5         | -67.58    | 0.9943    | 0.7256    | 175.9     |
| 6         | -66.23    | 0.9137    | 0.8985    | 149.6     |
| 7         | -66.55    | 0.9229    | 0.6774    | 183.7     |
| 8         | -66.7     | 0.8991    | 0.9124    | 143.2     |
| 9         | -65.82    | 0.6335    | 0.8915    | 192.9     |
| 10        | -66.29    | 0.8704    | 0.6397    | 186.7     |
| 11        | -66.42    | 0.7431    | 0.5939    | 113.8     |
| 12        | -66.83    | 0.7199    | 0.6706    | 165.3     |
| 13        | -66.05    | 0.6424    | 0.9809    | 192.1     |
| 14        | -67.71    | 0.5092    | 0.8948    | 196.1     |
| 15    

In [32]:
params = rf_bo.max['params']
params['n_estimators']= int(params['n_estimators'])
print(params)

{'max_features': 0.5502109586318007, 'max_samples': 0.6976186438466729, 'n_estimators': 152}
